<a href="https://colab.research.google.com/github/Thom-ar/Predicao_Cancer/blob/main/Pratica_ciencia_de_dados_para_pred_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ciência de Dados na Saúde: Criando Modelos para Predição do Câncer

**Professora:** Profa. Isadora Garcia Ferrão
📧 isadoraferrao@usp.br
🌐 [linkedin](https://www.linkedin.com/in/isadora-ferrao/)
🔗 [Instagram](https://www.instagram.com/isadora.ferrao/)

## # Case: Classificação de Câncer por meio de microRNA

**Descrição dos Dados:** Os dados foram coletados do [The Cancer Genome Atlas]() (TCGA), que é um programa internacional e de referência mundial de caracterização de mais de 33 tipos de câncer. Os dados são reais e foram devidamente anonimizados. Cada linha representa a amostra retirada de uma pessoa. As colunas são os tipos de microRNA e cada entrada representa a intensidade com que aquele microRNA está expresso. Os valores de expressão variam entre $[0, \infty]$. Valores próximos a zero indicam pouca expressão enquanto que o contrário indica uma alta expressão. Os dados também apresentam rótulos (veja o atributo class) sendo TP (primary solid tumor) indicando tumor e NT (normal tissue).


**Objetivo:** Construir um modelo para predizer quando uma pessoa tem câncer dado um exame de sequenciamento do RNA.

## Leitura dos Dados

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA



In [ ]:
df = pd.read_csv('brca_mirnaseq.csv', sep=';', header=0, decimal= ',' )
df

hsa.let.7a.1  hsa.let.7a.2  hsa.let.7a.3    hsa.let.7b   hsa.let.7c  \
0     8962.996542  17779.575039   9075.200383  24749.898857   341.298400   
1     7739.739862  15524.941906   7713.626636  23374.640471   801.487258   
2     8260.612670  16497.981335   8355.342958  10957.355911   635.811272   
3     9056.241254  18075.168478   9097.666150  26017.522731  2919.348415   
4    10897.303665  21822.338727  10963.956320  22204.253575  3313.009950   
..            ...           ...           ...           ...          ...   
837  10628.975280  21125.108661  10585.686678  23396.813364  3892.051211   
838  16799.785282  33603.904432  16883.338223  20731.006597  5263.331356   
839  13120.807001  26337.935723  13229.425112  18796.895124  6581.549565   
840   7979.531224  16006.280243   8106.687917  20462.010937  4040.296936   
841  10439.110392  20880.967721  10649.126224  17770.685685  1330.766196   

     hsa.let.7d   hsa.let.7e  hsa.let.7f.1  hsa.let.7f.2  hsa.let.7g  ...  \
0    406.164781  1470.179650     14.716795   3627.642977  387.417272  ...   
1    513.297924   560.962427     20.922042   6557.093894  350.955461  ...   
2    620.351816  2694.331127     39.799878  11830.760394  600.725980  ...   
3    334.245155  1322.434475     17.866463   6438.725384  354.957604  ...   
4    350.615669  1711.886682     22.541895   8246.117280  333.425447  ...   
..          ...          ...           ...           ...         ...  ...   
837  367.141461  1484.663795     23.402901  10570.535667  571.680109  ...   
838  201.676038  2173.283559     36.888271  18227.341203  870.301142  ...   
839  375.598820  2547.029500     28.505268  16838.042944  778.398745  ...   
840  295.594442   962.166120     23.885025   7625.121634  428.411748  ...   
841  790.868182  1952.822603     29.966587  10936.555740  577.855691  ...   

     hsa.mir.941.1  hsa.mir.942  hsa.mir.943  hsa.mir.944  hsa.mir.95  \
0              0.0     5.530515     0.187475     2.062226    4.124452   
1              0.0     8.180047     0.000000     0.629234    1.258469   
2              0.0     3.618171     0.000000     0.767491    1.644623   
3              0.0     3.478426     0.000000     3.478426    1.739213   
4              0.0     2.108235     0.000000     1.135203    0.810860   
..             ...          ...          ...          ...         ...   
837            0.0     1.217492     0.000000     0.405831    1.217492   
838            0.0     5.341744     0.000000     3.124416    2.318115   
839            0.0     1.863089     0.000000     0.558927    0.931545   
840            0.0     2.070956     0.000000     2.209020    1.656765   
841            0.0    11.487192     0.000000     3.745823    2.746937   

     hsa.mir.96  hsa.mir.98  hsa.mir.99a   hsa.mir.99b  class  
0    119.984057   53.992826   130.201449  46548.939810     TP  
1     60.249189   86.047798   236.434808  12644.149725     TP  
2     97.252043  117.645369   191.434123  33083.456616     TP  
3     72.572624   41.583007  1046.690127  24067.232290     TP  
4     19.947145   34.380445  1081.037952  25715.275426     TP  
..          ...         ...          ...           ...    ...  
837   79.813361   57.627952  1100.883277  16338.471420     TP  
838   16.629958   57.348159  1919.601107  14080.736733     TP  
839   41.919511   54.215901  1310.124456  17072.605898     TP  
840   55.225491   53.016472  1120.939408  18696.866174     TP  
841   44.949881   80.160621   470.225698  34080.000799     TP  

[842 rows x 898 columns]

In [ ]:
df.shape

(842, 898)

## Análise Exploratória dos Dados

In [ ]:
ax = sns.countplot(x='class', data=df)

In [ ]:
df['class'].value_counts()

class
TP    755
NT     87
Name: count, dtype: int64

In [ ]:
df['class'].value_counts(normalize=True)

class
TP    0.896675
NT    0.103325
Name: proportion, dtype: float64

In [ ]:
df.describe()

hsa.let.7a.1  hsa.let.7a.2  hsa.let.7a.3     hsa.let.7b    hsa.let.7c  \
count    842.000000    842.000000    842.000000     842.000000    842.000000   
mean    9218.938921  18432.504585   9289.250466   26606.604836   3152.699471   
std     4843.796136   9704.187427   4858.691217   16745.347957   3238.003201   
min     1294.149164   2599.981125   1319.952907    1817.920354    148.795934   
25%     5902.143848  11741.467528   5933.706564   14580.357100   1276.700850   
50%     8016.628565  16040.589880   8103.783439   23097.825936   2352.902327   
75%    11236.887034  22538.594950  11289.595988   34373.185504   3971.192192   
max    45101.697434  90233.655610  45095.490102  144706.427973  59677.212349   

        hsa.let.7d    hsa.let.7e  hsa.let.7f.1  hsa.let.7f.2   hsa.let.7g  \
count   842.000000    842.000000    842.000000    842.000000   842.000000   
mean    558.321269   1289.570177     24.359962   8687.461926   610.223836   
std     346.883205    763.056055     12.490091   6052.615278   317.854963   
min      79.783216    161.181457      2.439034    653.474578    88.614573   
25%     330.638301    809.867504     16.441786   4648.822942   410.859815   
50%     481.342371   1101.403395     21.890340   7019.157941   532.277053   
75%     681.931022   1619.864372     29.395515  10926.448322   724.277709   
max    3370.036117  11617.011618    121.408006  80780.055188  3342.745045   

       ...  hsa.mir.940  hsa.mir.941.1  hsa.mir.942  hsa.mir.943  hsa.mir.944  \
count  ...   842.000000     842.000000   842.000000   842.000000   842.000000   
mean   ...     5.902975       0.003737     6.446279     0.061018     2.320737   
std    ...     8.325681       0.049274     9.541682     0.172214     6.527536   
min    ...     0.000000       0.000000     0.000000     0.000000     0.000000   
25%    ...     1.378098       0.000000     2.464140     0.000000     0.373238   
50%    ...     3.192098       0.000000     4.127957     0.000000     1.036215   
75%    ...     7.159431       0.000000     7.551755     0.000000     2.345941   
max    ...    91.996543       0.909391   184.185656     1.757516   122.685820   

       hsa.mir.95  hsa.mir.96  hsa.mir.98   hsa.mir.99a    hsa.mir.99b  
count  842.000000  842.000000  842.000000    842.000000     842.000000  
mean     3.150482   38.307053   63.746405   1034.572148   44369.112203  
std      4.287594   33.791795   40.145314   1117.491608   32754.290751  
min      0.000000    0.000000    3.374223     18.400719    3475.079227  
25%      1.201951   14.906921   39.913493    387.430475   22769.094433  
50%      2.235731   29.634884   52.993693    710.026124   35594.670263  
75%      4.030888   51.258145   75.993914   1242.434228   53462.034662  
max     93.402785  259.127121  399.078716  15689.499524  248074.178531  

[8 rows x 897 columns]

## Estabelecendo um Baseline Comparativo

Antes de qualque modelagem vamos estabelecer um baseline, i.e., uma solução simples para o problema.

In [ ]:
x = df.drop('class', axis=1)
y = df['class']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [ ]:
y_train.value_counts(normalize=True)

class
TP    0.894737
NT    0.105263
Name: proportion, dtype: float64

In [ ]:
lr = LogisticRegression(random_state=42)
cv_list_lr_baseline = cross_val_score(
    lr,
    x_train,
    y_train,
    cv=10,
    scoring='balanced_accuracy'
)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
mean_cv_lr_baseline = np.mean(cv_list_lr_baseline)
std_cv_lr_baseline = np.std(cv_list_lr_baseline)
print(f"Performance (bac): {round(mean_cv_lr_baseline, 4)} +/- {round(std_cv_lr_baseline, 4)}")

Performance (bac): 0.9405 +/- 0.0379


## Modelagem

In [ ]:
knn = Pipeline(
    [
        ('mms', MinMaxScaler()),
        ('skb',SelectKBest(chi2,k=10)),
        ('knn', KNeighborsClassifier(
            n_neighbors=3,
            p=2,
            weights='uniform'
        ))
    ]
)
cv_list_knn_euclid = cross_val_score(
    knn,
    x_train,
    y_train,
    cv=10,
    scoring='balanced_accuracy'
)
mean_cv_knn_euclid = np.mean(cv_list_knn_euclid)
std_cv_knn_euclid = np.std(cv_list_knn_euclid)
print(f"Performance (bac): {round(mean_cv_knn_euclid, 4)} +/- {round(std_cv_knn_euclid, 4)}")

Performance (bac): 0.9952 +/- 0.0088


In [ ]:
knn = Pipeline(
    [
        ('mms', MinMaxScaler()),
        ('skb',SelectKBest(chi2,k=10)),
        ('knn', KNeighborsClassifier(
            n_neighbors=3,
            p=1,
            weights='uniform'
        ))
    ]
)
cv_list_knn_man = cross_val_score(
    knn,
    x_train,
    y_train,
    cv=10,
    scoring='balanced_accuracy'
)
mean_cv_knn_man = np.mean(cv_list_knn_man)
std_cv_knn_man = np.std(cv_list_knn_man)
print(f"Performance (bac): {round(mean_cv_knn_man, 4)} +/- {round(std_cv_knn_man, 4)}")

Performance (bac): 0.9881 +/- 0.0217


In [ ]:
lr = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(
            penalty='l2',
            C = 1,
            fit_intercept=True,
            class_weight= "balanced",
            random_state=42
        )
        )
    ]
)

cv_list_lr_l2 = cross_val_score(
    lr,
    x_train,
    y_train,
    cv=10,
    scoring="balanced_accuracy"
)
mean_cv_lr_l2 = np.mean(cv_list_lr_l2)
std_cv_lr_l2 = np.std(cv_list_lr_l2)
print(f"Performance (bac): {round(mean_cv_lr_l2, 4)} +/- {round(std_cv_lr_l2, 4)}")

Performance (bac): 0.9834 +/- 0.0231


In [ ]:
lr = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(
            penalty='l1',
            C = 1,
            fit_intercept=True,
            class_weight= "balanced",
            random_state=42,
            solver = 'liblinear'
        ))

    ]
)
cv_list_lr_l1 = cross_val_score(
    lr,
    x_train,
    y_train,
    cv=10,
    scoring="balanced_accuracy"
)

mean_cv_lr_l1 = np.mean(cv_list_lr_l1)
std_cv_lr_l1 = np.std(cv_list_lr_l1)
print(f"Performance (bac): {round(mean_cv_lr_l1, 4)} +/- {round(std_cv_lr_l1, 4)}")

Performance (bac): 0.9924 +/- 0.0084


In [ ]:
lr = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=10)),
        ('lr', LogisticRegression(
            penalty='l2',
            C = 1,
            fit_intercept=True,
            class_weight= "balanced",
            random_state=42
        ))
    ]
)
cv_list_lr_l2_pca = cross_val_score(
    lr,
    x_train,
    y_train,
    cv=10,
    scoring="balanced_accuracy"
)
mean_cv_lr_l2_pca = np.mean(cv_list_lr_l2_pca)
std_cv_lr_l2_pca = np.std(cv_list_lr_l2_pca)

print(f"Performance (bac): {round(mean_cv_lr_l2_pca, 4)} +/- {round(std_cv_lr_l2_pca, 4)}")

Performance (bac): 0.9648 +/- 0.038


## Avaliação Experimental

In [ ]:
# resultados da cross-validacao
df_results_cv = pd.DataFrame(
    [cv_list_lr_baseline, cv_list_knn_euclid, cv_list_knn_man, cv_list_lr_l2,cv_list_lr_l1, cv_list_lr_l2_pca],
    index=['lr_baseline', 'knn_euclid', 'knn_man', 'lr_l2', 'lr_l1', 'lr_l2_pca'],
).T

df_results_cv

lr_baseline  knn_euclid   knn_man     lr_l2     lr_l1  lr_l2_pca
0     0.916667    1.000000  1.000000  0.990566  0.990566   0.990566
1     0.907233    1.000000  1.000000  0.990566  0.990566   0.981132
2     0.916667    1.000000  1.000000  1.000000  1.000000   0.916667
3     0.981132    1.000000  1.000000  0.981132  0.990566   0.981132
4     0.990566    0.990566  0.990566  0.990566  0.990566   0.990566
5     0.897799    0.990566  0.990566  0.990566  0.990566   0.897799
6     0.916667    1.000000  1.000000  1.000000  1.000000   1.000000
7     0.971154    0.971154  0.971154  0.971154  0.971154   0.980769
8     1.000000    1.000000  0.928571  0.918956  1.000000   0.909341
9     0.907051    1.000000  1.000000  1.000000  1.000000   1.000000

In [ ]:
df_res = df_results_cv.stack().to_frame('balanced_accuracy')
df_res.index.rename(["fold","pipelines"], inplace=True)
df_res = df_res.reset_index()
df_res.head(12)


fold    pipelines  balanced_accuracy
0      0  lr_baseline           0.916667
1      0   knn_euclid           1.000000
2      0      knn_man           1.000000
3      0        lr_l2           0.990566
4      0        lr_l1           0.990566
5      0    lr_l2_pca           0.990566
6      1  lr_baseline           0.907233
7      1   knn_euclid           1.000000
8      1      knn_man           1.000000
9      1        lr_l2           0.990566
10     1        lr_l1           0.990566
11     1    lr_l2_pca           0.981132

In [ ]:
plt.figure(figsize=(10, 10))

ax = sns.boxplot(x="pipelines", y="balanced_accuracy", data=df_res)
ax = sns.swarmplot(x="pipelines", y="balanced_accuracy", data=df_res, color=".40")

In [ ]:
plt.figure(figsize=(10,10))

sns.catplot(x="pipelines", y="balanced_accuracy", kind="violin", data=df_res)

<Figure size 1000x1000 with 0 Axes>

## Vamos avaliar a performance final do nosso modelo

In [ ]:
# retreinar o pipeline selecionado com todos os dados de treinamento
lr = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=10)),
        ('lr', LogisticRegression(
            penalty='l2',
            C = 1,
            fit_intercept=True,
            class_weight= "balanced",
            random_state=42
        ))
    ]
)
cv_list_lr_l2_pca = cross_val_score(
    lr,
    x_train,
    y_train,
    cv=10,
    scoring="balanced_accuracy"
)
mean_cv_lr_l2_pca = np.mean(cv_list_lr_l2_pca)
std_cv_lr_l2_pca = np.std(cv_list_lr_l2_pca)

print(f"Performance (bac): {round(mean_cv_lr_l2_pca, 4)} +/- {round(std_cv_lr_l2_pca, 4)}")

Performance (bac): 0.9648 +/- 0.038


In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Ajustando a pipeline com os dados de treino
lr.fit(x_train, y_train)

# Exibindo a matriz de confusão usando a pipeline treinada
ConfusionMatrixDisplay.from_estimator(lr, x_test, y_test)
plt.show()


In [ ]:
ConfusionMatrixDisplay.from_estimator(lr, x_test, y_test, normalize='true')
plt.show()

## Referências & Links



1. [The Cancer Genome Atlas Program](https://www.cancer.gov/about-nci/organization/ccg/research/structural-genomics/tcga)
2. [Micro RNA](https://en.wikipedia.org/wiki/MicroRNA_sequencing)
3. [Sklearn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
